# RAG进阶：提升检索质量的实用技巧

本笔记演示三种提升RAG检索质量的核心技巧：
- **混合检索**：结合向量检索和关键词检索
- **重排序**：使用LLM对检索结果二次排序
- **元数据过滤**：基于文档属性精准过滤

环境要求：LangChain 1.0+, OpenAI API

## 环境准备

In [45]:
from dotenv import load_dotenv

import os

# 加载环境变量
load_dotenv()

True

## 准备示例文档

创建带元数据的示例文档用于演示

In [46]:
from langchain_core.documents import Document
import datetime

# 创建示例文档（带元数据）
documents = [
    Document(
        page_content="Python环境配置：首先安装Python 3.10以上版本，然后使用pip安装必要的依赖包。配置环境变量PYTHONPATH指向项目根目录。",
        metadata={"source": "tech_guide.md", "type": "tutorial", "year": 2024, "language": "zh"}
    ),
    Document(
        page_content="API密钥配置方法：在项目根目录创建.env文件，添加OPENAI_API_KEY=your_key_here。使用python-dotenv库加载环境变量。",
        metadata={"source": "api_docs.md", "type": "reference", "year": 2024, "language": "zh"}
    ),
    Document(
        page_content="2024年Q1销售额达到500万元，同比增长25%。主要增长来自企业客户，占总收入的60%。",
        metadata={"source": "sales_report.md", "type": "report", "year": 2024, "quarter": "Q1"}
    ),
    Document(
        page_content="2023年Q4销售额为400万元，环比增长15%。个人用户贡献了主要增长。",
        metadata={"source": "sales_report.md", "type": "report", "year": 2023, "quarter": "Q4"}
    ),
    Document(
        page_content="软件安装步骤：1. 下载安装包 2. 运行安装程序 3. 选择安装路径 4. 完成安装并重启系统。",
        metadata={"source": "user_manual.md", "type": "tutorial", "year": 2024, "language": "zh"}
    ),
    Document(
        page_content="Environment setup for development: Install Node.js 18+, configure npm registry, set up Git credentials.",
        metadata={"source": "dev_guide.md", "type": "tutorial", "year": 2024, "language": "en"}
    ),
    Document(
        page_content="数据库连接配置：在config.yaml中设置数据库URL、用户名和密码。支持MySQL、PostgreSQL等主流数据库。",
        metadata={"source": "tech_guide.md", "type": "reference", "year": 2024, "language": "zh"}
    ),
    Document(
        page_content="如何安装软件：首先确保系统满足最低配置要求，然后从官网下载对应版本的安装包，双击运行安装向导。",
        metadata={"source": "faq.md", "type": "tutorial", "year": 2024, "language": "zh"}
    )
]

print(f"准备了 {len(documents)} 个示例文档")

准备了 8 个示例文档


## 1. 基础向量检索（对比基准）

先创建一个基础的向量检索器作为对比基准

In [50]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 创建嵌入模型和向量库
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 其他供应商
embeddings = OpenAIEmbeddings(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="Qwen/Qwen3-Embedding-4B"
)
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="rag_advanced"
)

# 创建基础检索器
basic_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

print("✓ 基础向量检索器已创建")

✓ 基础向量检索器已创建


In [51]:
# 测试基础检索
query = "怎么装这个软件"
basic_results = basic_retriever.invoke(query)

print(f"查询：{query}")
print(f"\n基础检索结果（{len(basic_results)}个）：\n")
for i, doc in enumerate(basic_results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:80]}...\n")

查询：怎么装这个软件

基础检索结果（3个）：

1. [sales_report.md] 2023年Q4销售额为400万元，环比增长15%。个人用户贡献了主要增长。...

2. [sales_report.md] 2023年Q4销售额为400万元，环比增长15%。个人用户贡献了主要增长。...

3. [tech_guide.md] Python环境配置：首先安装Python 3.10以上版本，然后使用pip安装必要的依赖包。配置环境变量PYTHONPATH指向项目根目录。...



## 2. 混合检索：向量 + 关键词

结合向量检索（语义理解）和BM25关键词检索（精确匹配）

In [53]:
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# 创建BM25关键词检索器
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 3

# 创建混合检索器
hybrid_retriever = EnsembleRetriever(
    retrievers=[basic_retriever, bm25_retriever],
    weights=[0.6, 0.4]  # 向量60%，关键词40%
)

print("✓ 混合检索器已创建")

✓ 混合检索器已创建


In [54]:
# 测试混合检索
hybrid_results = hybrid_retriever.invoke(query)

print(f"查询：{query}")
print(f"\n混合检索结果（{len(hybrid_results)}个）：\n")
for i, doc in enumerate(hybrid_results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:80]}...\n")

查询：怎么装这个软件

混合检索结果（5个）：

1. [sales_report.md] 2023年Q4销售额为400万元，环比增长15%。个人用户贡献了主要增长。...

2. [tech_guide.md] Python环境配置：首先安装Python 3.10以上版本，然后使用pip安装必要的依赖包。配置环境变量PYTHONPATH指向项目根目录。...

3. [faq.md] 如何安装软件：首先确保系统满足最低配置要求，然后从官网下载对应版本的安装包，双击运行安装向导。...

4. [tech_guide.md] 数据库连接配置：在config.yaml中设置数据库URL、用户名和密码。支持MySQL、PostgreSQL等主流数据库。...

5. [dev_guide.md] Environment setup for development: Install Node.js 18+, configure npm registry, ...



## 3. 重排序：LLM二次精排

使用LLM对检索结果进行智能重排序和压缩

In [ ]:
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

# 创建重排序压缩器
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 其他供应商
model = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="deepseek-ai/DeepSeek-V3", temperature=0
)
compressor = LLMChainExtractor.from_llm(model)

# 将重排序应用到混合检索器
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=hybrid_retriever
)

print("✓ 重排序检索器已创建")

✓ 重排序检索器已创建


In [56]:
# 测试重排序检索
rerank_results = rerank_retriever.invoke(query)

print(f"查询：{query}")
print(f"\n重排序后结果（{len(rerank_results)}个）：\n")
for i, doc in enumerate(rerank_results, 1):
    print(f"{i}. [{doc.metadata['source']}]")
    print(f"   {doc.page_content}\n")

查询：怎么装这个软件

重排序后结果（1个）：

1. [faq.md]
   Extracted relevant parts: 如何安装软件：首先确保系统满足最低配置要求，然后从官网下载对应版本的安装包，双击运行安装向导。



## 4. 元数据过滤：精准定位

基于文档属性（类型、时间、语言等）过滤检索范围

In [66]:
# 创建带元数据过滤的检索器
filtered_retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 3,
        "filter": {
            "$and": [
                {"type": "tutorial"},
                {"language": "zh"}
            ]
        }
    }
)


# 测试元数据过滤
filtered_results = filtered_retriever.invoke("环境配置")

print("查询：环境配置（限定：中文教程）")
print(f"\n过滤后结果（{len(filtered_results)}个）：\n")
for i, doc in enumerate(filtered_results, 1):
    print(f"{i}. [{doc.metadata['source']}] type={doc.metadata['type']}, lang={doc.metadata['language']}")
    print(f"   {doc.page_content[:80]}...\n")

查询：环境配置（限定：中文教程）

过滤后结果（3个）：

1. [tech_guide.md] type=tutorial, lang=zh
   Python环境配置：首先安装Python 3.10以上版本，然后使用pip安装必要的依赖包。配置环境变量PYTHONPATH指向项目根目录。...

2. [tech_guide.md] type=tutorial, lang=zh
   Python环境配置：首先安装Python 3.10以上版本，然后使用pip安装必要的依赖包。配置环境变量PYTHONPATH指向项目根目录。...

3. [user_manual.md] type=tutorial, lang=zh
   软件安装步骤：1. 下载安装包 2. 运行安装程序 3. 选择安装路径 4. 完成安装并重启系统。...



In [67]:
# 时间过滤示例：只检索2024年的销售报告
sales_retriever = vector_store.as_retriever(
    search_kwargs={
        "k": 2,
        "filter": {
            "$and": [
                {"type": "report"},
                {"year": 2024}
            ]
        }
    }
)


sales_results = sales_retriever.invoke("销售数据")

print("查询：销售数据（限定：2024年报告）")
print(f"\n过滤后结果（{len(sales_results)}个）：\n")
for i, doc in enumerate(sales_results, 1):
    print(f"{i}. [{doc.metadata['source']}] year={doc.metadata['year']}, quarter={doc.metadata.get('quarter', 'N/A')}")
    print(f"   {doc.page_content}\n")

查询：销售数据（限定：2024年报告）

过滤后结果（2个）：

1. [sales_report.md] year=2024, quarter=Q1
   2024年Q1销售额达到500万元，同比增长25%。主要增长来自企业客户，占总收入的60%。

2. [sales_report.md] year=2024, quarter=Q1
   2024年Q1销售额达到500万元，同比增长25%。主要增长来自企业客户，占总收入的60%。



##  实用技巧总结

### 选择策略参考

| 场景 | 推荐方案 | 原因 |
|------|----------|------|
| 小规模文档库 (<10K) | 混合检索 + 元数据过滤 | 重排序成本不划算 |
| 大规模通用库 | 混合检索 + 重排序 | 两阶段策略平衡效率 |
| 结构化文档 | 元数据过滤 + 向量检索 | 先缩小范围再语义匹配 |
| 成本敏感 | 仅混合检索 | 避免LLM重排序成本 |

### 权重调优建议

- 专业术语多、代码多 → 关键词权重提高到 0.5
- 口语化查询多 → 向量权重提高到 0.7
- 平衡场景 → 默认 0.6/0.4

---

## 下一步

现在你已经掌握了提升检索质量的核心技巧。在下一个笔记中，我们将整合这些能力，构建一个完整的PDF问答机器人！